## 过滤垃圾邮件

In [2]:
import re
mySent = 'This book is the best book on Python or M.L. I have ever laid eyes upon.'
listOfTokens = re.split(r'\W+', mySent)
print(listOfTokens)
# tokenList = [tok for tok in listOfTokens if len(tok) > 0]
# tokenList = [tok.lower() for tok in listOfTokens]

['This', 'book', 'is', 'the', 'best', 'book', 'on', 'Python', 'or', 'M', 'L', 'I', 'have', 'ever', 'laid', 'eyes', 'upon', '']


### 创建词向量

In [52]:
def createVocabList(dataSet):
    """
    获取所有单词的集合
    :param dataSet: 数据集
    :return: 所有单词的集合(即不含重复元素的单词列表)
    """
    vocabSet = set([])
    # create empty set
    for document in dataSet:
        # 操作符 | 用于求两个集合的并集
        vocabSet = vocabSet | set(document)
        # union of the two sets
    return list(vocabSet)


def setOfWords2Vec(vocabList, inputSet):
    """
    遍历查看该单词是否出现，出现该单词则将该单词置1
    :param vocabList: 所有单词集合列表
    :param inputSet: 输入数据集
    :return: 匹配列表[0,1,0,1...]，其中 1与0 表示词汇表中的单词是否出现在输入的数据集中
    """
    # 创建一个和词汇表等长的向量，并将其元素都设置为0
    returnVec = [0] * len(vocabList) # [0,0......]
    # 遍历文档中的所有单词，如果出现了词汇表中的单词，则将输出的文档向量中的对应值设为1
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:
            print ("the word: %s is not in my Vocabulary!" %word)
    return returnVec

In [11]:
from numpy import *

In [57]:
def trainNB0(trainMatrix, trainCategory):
    """
    训练数据优化版本
    :param trainMatrix: 文件单词矩阵
    :param trainCategory: 文件对应的类别
    :return:
    """
    # 总文件数
    numTrainDocs = len(trainMatrix)
    # 总单词数
    numWords = len(trainMatrix[0])
    # 侮辱性文件的出现概率
    pAbusive = sum(trainCategory) / float(numTrainDocs)
    # 构造单词出现次数列表
    # p0Num 正常的统计
    # p1Num 侮辱的统计
    p0Num = ones(numWords)#[0,0......]->[1,1,1,1,1.....]
    p1Num = ones(numWords)
    # 整个数据集单词出现总数，2.0根据样本/实际调查结果调整分母的值（2主要是避免分母为0，当然值可以调整）
    # p0Denom 正常的统计
    # p1Denom 侮辱的统计
    p0Denom = 2.0
    p1Denom = 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            # 累加辱骂词的频次
            p1Num += trainMatrix[i]
            # 对每篇文章的辱骂的频次 进行统计汇总
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
            # 类别1，即侮辱性文档的[log(P(F1|C1)),log(P(F2|C1)),log(P(F3|C1)),log(P(F4|C1)),log(P(F5|C1))....]列表
            p1Vect = log(p1Num / p1Denom)
            # 类别0，即正常文档的[log(P(F1|C0)),log(P(F2|C0)),log(P(F3|C0)),log(P(F4|C0)),log(P(F5|C0))....]列表
            p0Vect = log(p0Num / p0Denom)
    return p0Vect, p1Vect, pAbusive


In [ ]:
### 

In [62]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    """
    使用算法：
    # 将乘法转换为加法
    乘法：P(C|F1F2...Fn) = P(F1F2...Fn|C)P(C)/P(F1F2...Fn)
    加法：P(F1|C)*P(F2|C)....P(Fn|C)P(C) -> log(P(F1|C))+log(P(F2|C))+....+log(P(Fn|C))+log(P(C))
    :param vec2Classify: 待测数据[0,1,1,1,1...]，即要分类的向量
    :param p0Vec: 类别0，即正常文档的[log(P(F1|C0)),log(P(F2|C0)),log(P(F3|C0)),log(P(F4|C0)),log(P(F5|C0))....]列表
    :param p1Vec: 类别1，即侮辱性文档的[log(P(F1|C1)),log(P(F2|C1)),log(P(F3|C1)),log(P(F4|C1)),log(P(F5|C1))....]列表
    :param pClass1: 类别1，侮辱性文件的出现概率    :return: 类别1 or 0
    """
    # 计算公式  log(P(F1|C))+log(P(F2|C))+....+log(P(Fn|C))+log(P(C))
    # 大家可能会发现，上面的计算公式，没有除以贝叶斯准则的公式的分母，也就是 P(w) （P(w) 指的是此文档在所有的文档中出现的概率）就进行概率大小的比较了，
    # 因为 P(w) 针对的是包含侮辱和非侮辱的全部文档，所以 P(w) 是相同的。
    # 使用 NumPy 数组来计算两个向量相乘的结果，这里的相乘是指对应元素相乘，即先将两个向量中的第一个元素相乘，然后将第2个元素相乘，以此类推。
    # 我的理解是：这里的 vec2Classify * p1Vec 的意思就是将每个词与其对应的概率相关联起来
    p1 = sum(vec2Classify * p1Vec) + log(pClass1) # P(w|c1) * P(c1) ，即贝叶斯准则的分子
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1) # P(w|c0) * P(c0) ，即贝叶斯准则的分子·
    if p1 > p0:
        return 1
    else:
        return 0

In [61]:
# 切分文本
def textParse(bigString):
    '''
    Desc:        接收一个大字符串并将其解析为字符串列表
    Args:        bigString -- 大字符串
    Returns:        去掉少于 2 个字符的字符串，并将所有字符串转换为小写，返回字符串列表
    '''
    import re
    # 使用正则表达式来切分句子，其中分隔符是除单词、数字外的任意字符串
    listOfTokens = re.split(r'\W+', bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]


def spamTest():
    '''
    Desc:        对贝叶斯垃圾邮件分类器进行自动化处理。
    Args:        none
    Returns:        对测试集中的每封邮件进行分类，若邮件分类错误，则错误数加 1，最后返回总的错误百分比。
    '''
    docList = []
    classList = []
    for i in range(1, 26):
        # 切分，解析数据，并归类为 1 类别 
        wordList = textParse(open('email/spam/%d.txt' % i, 'rb').read().decode('utf8','ignore'))
        docList.append(wordList)
        classList.append(1)
        # 切分，解析数据，并归类为 0 类别 垃圾邮件
        wordList = textParse(open('email/ham/%d.txt' % i, 'rb').read().decode('utf8','ignore'))
        docList.append(wordList)
        classList.append(0)
    
    # 创建词汇表
    vocabList = createVocabList(docList)
    trainingSet = list(range(50))
    testSet = []
    
    # 随机取 10 个邮件用来测试
    for _ in range(10):
        # random.uniform(x, y) 随机生成一个范围为 x - y 的实数
        randIndex = int(random.uniform(0, len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])
        
    trainMat = []
    trainClasses = []
    for docIndex in trainingSet:
        trainMat.append(setOfWords2Vec(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V, p1V, pSpam = trainNB0(array(trainMat), array(trainClasses))

    errorCount = 0
    for docIndex in testSet:
        wordVector = setOfWords2Vec(vocabList, docList[docIndex])
        if classifyNB(array(wordVector), p0V, p1V, pSpam) != classList[docIndex]:
            errorCount += 1
    
    print('the errorCount is: ', errorCount)
    print('the testSet length is :', len(testSet))
    print('the error rate is :', float(errorCount)/len(testSet))



In [63]:
spamTest()

the errorCount is:  0
the testSet length is : 10
the error rate is : 0.0


In [58]:
# open('email/spam/17.txt', 'rb').read().decode('utf8','ignore')
# textParse(open('email/ham/6.txt', 'rb').read().decode('utf8','ignore'))

In [64]:
vocabList

NameError: name 'vocabList' is not defined

In [68]:
docList = []
classList = []
for i in range(1, 26):
    # 切分，解析数据，并归类为 1 类别 
    wordList = textParse(open('email/spam/%d.txt' % i, 'rb').read().decode('utf8','ignore'))
    docList.append(wordList)
    classList.append(1)
    
    # 切分，解析数据，并归类为 0 类别 垃圾邮件
    wordList = textParse(open('email/ham/%d.txt' % i, 'rb').read().decode('utf8','ignore'))
    docList.append(wordList)
    classList.append(0)
    
# 创建词汇表
vocabList = createVocabList(docList)

In [69]:
vocabList

['changes',
 'answer',
 'finance',
 'gain',
 'bathroom',
 'invitation',
 'wilson',
 'location',
 'museum',
 'insights',
 '513',
 'borders',
 'note',
 'office',
 'storage',
 'logged',
 'hours',
 '120',
 'pretty',
 'with',
 'out',
 'cheap',
 'items',
 'questions',
 'carlo',
 'automatically',
 'ems',
 'mail',
 '25mg',
 'fans',
 'your',
 'top',
 'giants',
 'fractal',
 'worldwide',
 'going',
 'cannot',
 'source',
 'expo',
 '588',
 'thread',
 'two',
 'only',
 'computer',
 'pages',
 'sky',
 'died',
 'holiday',
 'done',
 'dusty',
 'experts',
 '66343',
 'specifically',
 'assistance',
 'endorsed',
 'ofejacu1ate',
 'owner',
 'brandviagra',
 'regards',
 'window',
 'meet',
 'treat',
 'upload',
 'thank',
 'cats',
 'jquery',
 'another',
 'brands',
 'who',
 'starting',
 'hermes',
 'find',
 'pick',
 'support',
 'butt',
 'express',
 'naturalpenisenhancement',
 'call',
 'files',
 '50092',
 'through',
 'past',
 '292',
 '396',
 'free',
 'julius',
 'mathematics',
 'inches',
 'now',
 'writing',
 'chapter',
 

In [70]:
# 创建词汇表
vocabList = createVocabList(docList)
trainingSet = list(range(50))
testSet = []

# 随机取 10 个邮件用来测试
for _ in range(10):
    # random.uniform(x, y) 随机生成一个范围为 x - y 的实数
    randIndex = int(random.uniform(0, len(trainingSet)))
    testSet.append(trainingSet[randIndex])
    del(trainingSet[randIndex])

trainMat = []
trainClasses = []
for docIndex in trainingSet:
    trainMat.append(setOfWords2Vec(vocabList, docList[docIndex]))
    trainClasses.append(classList[docIndex])
p0V, p1V, pSpam = trainNB0(array(trainMat), array(trainClasses))

In [73]:
pSpam

0.475